# Problem
You are in front of a unstable rope bridge, with 3 others waiting to cross your way to safety on a dark night. You have limited time to cross and each of you need different amount of time to get across.

- You: 1 min
- Assistant: 2 min
- Janitor: 5 min
- Prof: 10 min

The bridge being rickety cannot allow more than 2 on it at any given time. Also because it is dark, you need to use the only torch which you have to share among yourselves.

What is the minimum time needed? Can you get across in 17 minutes?

This is based on Alex Gendler's description: https://www.youtube.com/watch?v=7yDmGnA8Hw0

In [1]:
travelTime = {"Prof":10, "Janitor": 5, "Asst": 2, "Me": 1} 

# Solution

The trick to such problems is always to get the decision variables right!

## Trips
Since, there are 4 people, the minimum no. of trips needed in any case is 3, with the first two trips followed by, a person coming back with the torch

In [2]:
Ntrips = len(travelTime) - 1

### Packages

In [3]:
import pulp as pl

## Decision variables

For each person, define binary decision variables, corresponding to each of the
- 3 onward trips, and 
- 2 return trips

If the person goes across, the corresponding binary variable would switch on (1), otherwise, it will be off (0).

Additionally, define waiting variables after each trip, across the bridge

In [4]:
X1 = pl.LpVariable.dict( "Trip 1", travelTime.keys(), cat = pl.LpBinary)
Y1 = pl.LpVariable.dict("Trip 1 Rt", travelTime.keys(), cat = pl.LpBinary)
W1 = pl.LpVariable.dict( "Wait 1", travelTime.keys(), cat = pl.LpBinary)

X2 = pl.LpVariable.dict("Trip 2", travelTime.keys(), cat = pl.LpBinary)
Y2 = pl.LpVariable.dict("Trip 2 Rt", travelTime.keys(), cat = pl.LpBinary)
W2 = pl.LpVariable.dict( "Wait 2", travelTime.keys(), cat = pl.LpBinary)

X3 = pl.LpVariable.dict("Trip 3", travelTime.keys(), cat = pl.LpBinary )

## Initialize the problem

In [5]:
prob = pl.LpProblem("Lab zombie", pl.LpMinimize)

## Constraints

### Trip 1

- Only 2 of the people should go on the bridge

In [6]:
prob += pl.lpSum(X1) == 2, "Trip 1 onward"

### Return from Trip 1

- 1 of those on the other side should return with the torch

In [7]:
prob += pl.lpSum(Y1) == 1, "Trip 1 returns"

- While the above ensures that one of the 4 people return, how do we ensure that it is only one of those who got across the bridge in the first place?
- Ensure that the return variablesof each person doesn't exceed its corresponding onward trip value!

In [8]:
for p in travelTime:
    prob += Y1[p] <= X1[p], "Trip 1 valid return of " + str(p)

### Waiting across the bridge after Trip 1

- The persons waiting across the bridge would have their corresponding waiting variable on (1)
- Waiting variable corresponds to those who came across and did not return
    - Difference between onward and return trip variable

In [9]:
for p in travelTime:
    prob += W1[p] == X1[p]- Y1[p], "Trip 1 waiting " + str(p)

### Trip 2
- Only 2 can go on the bridge, but they can be one among:
    - those still waiting on the departure side of the bridge (did not go across on the first trip)
    - the one who returned after the first trip with the torch

- Only 2 can go at a time

In [10]:
prob += pl.lpSum(X2) == 2, "Trip 2 onward"

- Ensure that the one who is waiting on the other side, is excluded from the second trip
    - the corresponding second trip variable should be zero always
        - For the waiting person, from the first trip, difference between onward and return variable must be 1
        - For all others, this difference in the first trip variables is 0

In [11]:
for p in travelTime:
    prob += X2[p] <= 1 - W1[p], "Trip 2 valid onward " + str(p)

### Trip 2 return
- Only one can return, from among 3 people
    - those who got across on the second trip
    - the one waiting from the first strip

- Only one can return

In [12]:
prob += pl.lpSum(Y2) == 1, "Trip 2 returns"

- Ensure those who return are either 
    - those who participated in the second trip (second trip variables)
    - the one who was waiting all along across the bridge

In [13]:
for p in travelTime:
    # For each person, ensure
    #   they can only return if, 
    #               they just went across on the second trip
    #                      or, they were waiting after the first trip
    prob += Y2[p] <= X2[p] + W1[p], "Trip 2 valid return of " + str(p)

### Waiting across the bridge after Trip 2

- The persons waiting across the bridge would have their corresponding waiting variable on (1)
- Waiting variable corresponds to those who came across now and did not return
    - Difference between onward and return trip variable
    - Or, who were waiting already from the first trip

In [14]:
for p in travelTime:
    prob += W2[p] == W1[p] + X2[p]- Y2[p], "Trip 2 waiting " + str(p)

### Trip 3
- Only those 2 can participate, who is not across the bridge already

- Only 2 can participate

In [15]:
prob += pl.lpSum(X3) == 2, "Trip 3 onward"

- Ensure the one/s waiting across the bridge don't participate
    - the corresponding third trip variable should be zero always
        - For the waiting person/s, from the first or second trip, difference between onward and return variable will be 1        
        - For all others, this difference in the second trip variables is 0

In [16]:
for p in travelTime:
    prob += X3[p] <= 1 - W2[p], "Trip 3 valid onward " + str(p)

*Trip 3 has no return journey*

### Safety concern
- Each person must at least once go across the bridge

for p in travelTime:
    prob += X1[p] + X2[p] + X3[p] >= 1, str(p) + " must get across"

## Travel time

- Each trip's time is the maximum of the ones taking the trip
- We can create variables for onward trip time
    - For each trip, ensure the trip-time is greater than the time of those taking the trip

In [17]:
t1 = pl.LpVariable('t1', lowBound=0, cat = pl.LpContinuous)
t2 = pl.LpVariable('t2', lowBound=0, cat = pl.LpContinuous)
t3 = pl.LpVariable('t3', lowBound=0, cat = pl.LpContinuous)

In [18]:
for p in travelTime:
    prob += t1 >= X1[p]*travelTime[p], "Trip 1 time for " + str(p)
    prob += t2 >= X2[p]*travelTime[p], "Trip 2 time for " + str(p)
    prob += t3 >= X3[p]*travelTime[p], "Trip 3 time for " + str(p)

## Minimize the travel time

- Total time for the entire onwards and return trip

In [19]:
prob += t1 + pl.lpDot(Y1.values(), travelTime.values()) + t2 + pl.lpDot(Y2.values(), travelTime.values()) + t3 , "Total travel time"

## Results

In [20]:
prob.solve()

1

In [21]:
len(prob.variables())

31

In [29]:
len(travelTime) * 3 + len(travelTime) * 3 + len(travelTime) + 3

31

In [22]:
prob.objective.value()

17.0

In [23]:
if prob.status > 0:
    print("Time taken to cross:", prob.objective.value())
    for var in prob.variables():
        if var.value() > 0 and var.cat == pl.LpInteger and "Trip" in var.name:
            print("\t", var)
else: print("Problem infeasible")

Time taken to cross: 17.0
	 Trip_1_Asst
	 Trip_1_Me
	 Trip_1_Rt_Me
	 Trip_2_Janitor
	 Trip_2_Prof
	 Trip_2_Rt_Asst
	 Trip_3_Asst
	 Trip_3_Me


In [24]:
tripTravellers = {}
trip = 1
tripTravellers["Trip " + str(trip)] = {"Onward":[X1[p] for p in travelTime if X1[p].value() > 0], 
                                       "Return":[Y1[p] for p in travelTime if Y1[p].value() > 0],
                                      "Time": t1.value() + pl.lpDot(Y1.values(), travelTime.values()).value()}
trip = 2
tripTravellers["Trip " + str(trip)] = {"Onward":[X2[p] for p in travelTime if X2[p].value() > 0], 
                                       "Return":[Y2[p] for p in travelTime if Y2[p].value() > 0],
                                       "Time": t2.value() + pl.lpDot(Y2.values(), travelTime.values()).value()}
trip = 3
tripTravellers["Trip " + str(trip)] = {"Onward":[X3[p] for p in travelTime if X3[p].value() > 0],
                                       "Time": t3.value()}

In [25]:
t1.value(), t2.value(), t3.value()

(2.0, 10.0, 2.0)

In [26]:
tripTravellers

{'Trip 1': {'Onward': [Trip_1_Asst, Trip_1_Me],
  'Return': [Trip_1_Rt_Me],
  'Time': 3.0},
 'Trip 2': {'Onward': [Trip_2_Prof, Trip_2_Janitor],
  'Return': [Trip_2_Rt_Asst],
  'Time': 12.0},
 'Trip 3': {'Onward': [Trip_3_Asst, Trip_3_Me], 'Time': 2.0}}